Imports

In [1]:
# Import necessary libraries
from nnsight import LanguageModel
import torch
import pandas as pd

# Initialize the language model with specific parameters
model = LanguageModel("google/gemma-2b-it", trust_remote_code=True, device_map="cuda:0", low_cpu_mem_usage=True, torch_dtype=torch.float16)
model.requires_grad_(False)



from sae_lens import SAE
layer = 6

# get the SAE for this layer
sae, cfg_dict, _ = SAE.from_pretrained(
    release = "gemma-2b-res-jb",
    sae_id = f"blocks.{layer}.hook_resid_post",
    device = 'cuda:0'
)



Parameters

In [48]:
#@title Self-explanation in 18 lines
feature = 471  #@param {type: "integer"}
#feature = 5892  #@param {type: "integer"}
# feature = 9415
scale = 76  #@param {type: "number"}
se_demo = True  #@param {type: "boolean"}
max_new_tokens = 80  #@param {type: "integer"}
n_generate = 1  #@param {type: "integer"}
vals = ['positive', 'neutral', 'negative']
samples = pd.read_excel('/home/ashater/work/sae/sentence_sentiment_gpt4o.xlsx')

## Main code prompt


In [37]:


v = ["How does does financial sentiment affect credit rating, particular related to stock price?", "what is IBM credit rating and how does  it effect lending?"]

In [51]:
full_res = []
#v = range(0,10)
for ismpl in v:


    # prompt = """<start_of_turn>user\nCan you rate  the following sentnece as a financial credit sentiment, very positive, 
    # somewhat positive, neutral, somewhat negative, 
    # very negative? """ + samples.iloc[ismpl].Sentence + '<end_of_turn>\n<start_of_turn>model\nThe sentiment of the setence is "X"'

    prompt = '<start_of_turn>user\nWhat is the meaning of the word "X"?<end_of_turn>\n<start_of_turn>model\nThe meaning of the word "X" is "'

    # prompt = '<start_of_turn>user\n' + ismpl +  '<end_of_turn>\n<start_of_turn>model X'
    positions = [i for i, a in enumerate(model.tokenizer.encode(prompt)) if model.tokenizer.decode([a]) == "X"]
    with model.generate(prompt, max_new_tokens=max_new_tokens, num_return_sequences=n_generate, do_sample=True, scan=False, validate=False) as gen:
        # vector steering section
        vector = sae.W_dec[[feature]]
        vector = vector / vector.norm()
        vector = vector * scale
        # setting vector into layer 2
        for position in positions:
          model.model.layers[2].output[0][:, position] = vector
            
        # get final output saved
        out = model.generator.output.save()
        # capture at layer 6 or 12
        resid = model.model.layers[12].output.save()
        
    features = sae.encode(resid[0])
    summed_activations = features.abs().sum(dim=1) # Sort by max activations
    top_activations_indices = summed_activations.topk(20).indices # Get indices of top 20    

    for i, l in enumerate(model.tokenizer.batch_decode(out)):
        s = repr(l.partition(prompt)[2].partition("<eos>")[0])
        print(prompt + "|"+ s + "|" + str(top_activations_indices))
        # print(str(top_activations_indices))


<start_of_turn>user
What is the meaning of the word "X"?<end_of_turn>
<start_of_turn>model
The meaning of the word "X" is "|'a measure of the creditworthiness and strength of a person, organization, or country". It is often used to indicate the probability of a borrower or issuer fulfilling their financial obligations.'|tensor([[  471, 11912, 10771,  3390,  1213, 15881,  5347,  2157, 16334,   556,
         11785, 15299, 12670,  1630,  5490,  3586,  1587, 12630,  5624,  5396]],
       device='cuda:0')
<start_of_turn>user
What is the meaning of the word "X"?<end_of_turn>
<start_of_turn>model
The meaning of the word "X" is "|'a measure of the creditworthiness, financial standing, or operating capacity of an entity". It is often used by investors, creditors, and other parties who make decisions based on this information.\n\nIn a financial context, a credit rating is a numerical rating given to a company or other entity by a major credit rating agency, such as Moody\'s Investors Service or 

## summed activations across tokens

In [30]:
summed_activations.shape

torch.Size([1, 16384])

## Get features for each token

In [36]:
features.shape

torch.Size([1, 21, 16384])